# Segmenting and Clustering Neighborhoods in Toronto - First Part

In this Notebook we create a new Notebook and use it to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

#### First, we import the libraries we expect to need:


We need to know the Python version we are using to install beautifulsoup

In [1]:
from platform import python_version
print(python_version())

3.6.10


In [2]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 7.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install lxml

     |████████████████████████████████| 5.8MB 5.4MB/s eta 0:00:015.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#### Then, we download the data from Wikipedia 

In [3]:
website_link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_link,'lxml')
#print(soup.prettify())

And we extract the information from it

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

In [5]:
My_table2 = soup.find('table')
#My_table2

In [6]:
links = My_table2.findAll('tr')
#links

In [7]:
table_rows = My_table2.find_all('tr')
import pandas as pd
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
#data

Now that we have the information like this:
[[],
 ['M1A', 'Not assigned', ''],
 ['M2A', 'Not assigned', ''],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],...
we can transform the data into a pandas dataframe. We set 3 columns and name them and filter out bad rows and take a look at the head.

In [8]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


We discard Not assigned Boroughs:

In [9]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


And reset the index:

In [10]:
df1 = df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Regent Park / Harbourfront
3,6,M6A,North York,Lawrence Manor / Lawrence Heights
4,7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          103 non-null    int64 
 1   PostalCode     103 non-null    object
 2   Borough        103 non-null    object
 3   Neighbourhood  103 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [12]:
df1.shape

(103, 4)

We combine neighbourhoods with the same PostalCode into one row, with the neighborhoods by commas

In [13]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Borough        103 non-null    object
 1   Neighbourhood  103 non-null    object
dtypes: object(2)
memory usage: 2.4+ KB


For Neighborhoods as Not assigned, we substitute with the corresponding Borough

In [15]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


We add index:

In [16]:
df3 = df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df4=df3
#Now we can remove the duplicate boroughts as follows:
df4['Borough']= df4['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df4.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostalCode     103 non-null    object
 1   Borough        103 non-null    object
 2   Neighbourhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [19]:
df4.shape

(103, 3)

Neighborhoods are separated by "/" instead of comma, we can replace them:

In [20]:
df5=df4
df5['Neighbourhood']= df5['Neighbourhood'].str.replace('/', ',')
df5.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finally, we can save the dataframe as csv (for later) and check the final dataframe shape

In [22]:
df5.to_csv('data_Toronto.csv')

In [24]:
df4.shape

(103, 3)